In [3]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as stp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [4]:
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')
df_cp = df.copy()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [7]:
df_names = df['name'].copy()

In [ ]:
df.drop(['accomplishments_segment','education_segment','emails','misc_segment','objectives_segment','phone','projects_segment','skills_segment','text',
         'university_0','university_1','university_2','university_3','university_4','university_5','url','work_segment'],axis=1,inplace=True)
df.shape

In [ ]:
print('After removing the columns from dataset..')
df.isnull().sum()

In [10]:
df['degree'].fillna(df.degree.mode()[0],inplace=True)
df['links'].fillna('Missing',inplace=True)
df['work_experience'].fillna(0,inplace=True)
df['job_titles'].fillna('No Job',inplace=True)

In [ ]:
all_degrees = ''
for i in df.degree:
  if len(all_degrees) == 0:
    all_degrees = i
  else:
    all_degrees = all_degrees + ' , ' + i

all_degrees = all_degrees.split(',')
all_degrees = [re.sub("[\s.]","",i).upper() for i in all_degrees]
unique_degrees = set(all_degrees)
unique_degrees

In [12]:
df['bachelor_degrees'] = 'No Degree'
df['master_degrees'] = 'No Degree'
df['docterte_degrees'] = 'No Degree'
df['profiles'] = 'No Profile'

In [13]:
ind = 0
for i in df.degree:
  lst = re.sub("[\s.]","",i).upper( )
  # print(lst)
  for j in lst.split(','):
    if j in ['BE', 'BS', 'BSC', 'BTECH']:
      if df.loc[ind,'bachelor_degrees'] == 'No Degree':
        df.loc[ind,'bachelor_degrees'] = j
      else:
        df.loc[ind,'bachelor_degrees'] = df.loc[ind,'bachelor_degrees'] + ' , '+ j
    elif j in ['ME', 'MS', 'MSC', 'MTECH']:
      if df.loc[ind,'master_degrees'] == 'No Degree':
        df.loc[ind,'master_degrees'] = j
      else:
        df.loc[ind,'master_degrees'] = df.loc[ind,'master_degrees'] + ' , '+ j
    elif j == 'PHD':
      if df.loc[ind,'docterte_degrees'] == 'No Degree':
        df.loc[ind,'docterte_degrees'] = j
      else:
        df.loc[ind,'docterte_degrees'] = df.loc[ind,'docterte_degrees'] + ' , '+ j
  ind +=1

In [ ]:
df
# df.work_experience.value_counts()
# df.drop(['degree','links'],axis=1,inplace=True)
# df.isnull().sum()

In [15]:
ind = 0
for i in df.links:
  lst = re.sub("[\s]","",i)
  # print(lst)
  for j in lst.split(','):
    if j.find('github') != -1:
      df.loc[ind,'profiles'] = 'Github'
    elif j.find('linkedin') != -1:
      if df.loc[ind,'profiles'] == 'No Profile':
        df.loc[ind,'profiles'] = 'Linkedin'
      else:
        df.loc[ind,'profiles'] = df.loc[ind,'profiles'] + ' , ' + 'Linkedin'
  ind +=1

In [16]:
ind = 0
for i in df.work_experience:
  if i < 0:
    df.loc[ind,'work_experience'] = i*-1
  elif i > 40:
    df.loc[ind,'work_experience'] = 0
  ind +=1
df['work_experience'] = df['work_experience'].astype(int)

In [17]:
with open('/content/drive/MyDrive/Job Description.txt','r',encoding = 'utf-8') as f:
  file_desc_lst =  [r.replace('\n', '') for r in f.readlines()]

In [ ]:
job_description = ''

for i in file_desc_lst:
  if len(job_description) == 0:
    job_description = str(i)
  else:
    job_description = job_description + ' ' + str(i)
print(job_description)

In [19]:
all_resume_text = []

for i in df.iloc[:].values:
  s = ''
  for j in list(i):
    if len(s) == 0:
      s = str(j)
    else:
      s = s + ' , ' + str(j)
  all_resume_text.append(s)

In [20]:
lemmatizer = WordNetLemmatizer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in analyzer(doc) if w not in set(stp.words('english')))

In [21]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [22]:
def get_tf_idf_cosine_similarity(job_desc, all_resumes):
    tf_idf_vect = TfidfVectorizer(analyzer=stemmed_words)
    job_description = [job_desc]
    tf_idf_desc_vector = np.array(tf_idf_vect.fit_transform(job_description).todense())
    tf_idf_desc_vector = tf_idf_desc_vector.reshape(1, -1)
    tf_idf_resume_vector = np.array(tf_idf_vect.transform(all_resumes).todense())
    cosine_similarity_list = []
    for resume_vector in tf_idf_resume_vector:
      cosine_similarity_list.append(cosine_similarity(tf_idf_desc_vector, resume_vector.reshape(1,-1))[0][0])
    return cosine_similarity_list

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [24]:
import numpy as np

In [25]:
cos_sim_list = get_tf_idf_cosine_similarity(job_description, all_resume_text)

In [26]:
zipped_resume_rating = zip(cos_sim_list,df_cp.name,[x for x in range(len(df))])
sorted_resume_rating_list = sorted(zipped_resume_rating, key = lambda x: x[0], reverse=True)

In [ ]:
resume_score = [round(x*100,2) for x in cos_sim_list]
pd.concat([df_cp.name,pd.DataFrame(resume_score,columns=['resume_score(%)'])],axis=1).sort_values(by=['resume_score(%)'],ascending=False).head(10)


In [28]:
# Encoding categorical variables
label_encoder = LabelEncoder()
df['bachelor_degrees'] = label_encoder.fit_transform(df['bachelor_degrees'])
df['master_degrees'] = label_encoder.fit_transform(df['master_degrees'])
df['docterte_degrees'] = label_encoder.fit_transform(df['docterte_degrees'])
df['profiles'] = label_encoder.fit_transform(df['profiles'])

# Extracting features and target variable
X = df[['bachelor_degrees', 'master_degrees', 'docterte_degrees', 'profiles', 'work_experience']]
y = cos_sim_list  # Using cosine similarity as the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [ ]:
# Encoding categorical variables
label_encoder = LabelEncoder()
df['bachelor_degrees'] = label_encoder.fit_transform(df['bachelor_degrees'])
df['master_degrees'] = label_encoder.fit_transform(df['master_degrees'])
df['docterte_degrees'] = label_encoder.fit_transform(df['docterte_degrees'])
df['profiles'] = label_encoder.fit_transform(df['profiles'])

# Extracting features and target variable
X = df[['bachelor_degrees', 'master_degrees', 'docterte_degrees', 'profiles', 'work_experience']]
y = cos_sim_list  # Using cosine similarity as the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results_df.head(10))
resume_score = [round(x*100,2) for x in cos_sim_list]
pd.concat([df_cp.name,pd.DataFrame(resume_score,columns=['resume_score(%)'])],axis=1).sort_values(by=['resume_score(%)'],ascending=False).head(10)


In [ ]:
!pip install gradio type-extensions


In [ ]:
!pip install gradio

In [ ]:
!pip install pydantic

In [2]:
from sklearn.metrics import r2_score
!pip install gradio
import gradio as gr

In [39]:
def get_resume_score(job_description, df, all_resume_text):
    cos_sim_list = get_tf_idf_cosine_similarity(job_description, all_resume_text)

    # Encode categorical variables
    label_encoder = LabelEncoder()
    df['bachelor_degrees'] = label_encoder.fit_transform(df['bachelor_degrees'])
    df['master_degrees'] = label_encoder.fit_transform(df['master_degrees'])
    df['docterte_degrees'] = label_encoder.fit_transform(df['docterte_degrees'])
    df['profiles'] = label_encoder.fit_transform(df['profiles'])

    # Extract features and target variable
    X = df[['bachelor_degrees', 'master_degrees', 'docterte_degrees', 'profiles', 'work_experience']]
    y = cos_sim_list

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the Random Forest Regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = rf_model.predict(X_test)

    # Calculate Mean Squared Error
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')
    results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    print(results_df.head(10))

    # Calculate R-squared
    r2 = r2_score(y_test, y_pred)
    accuracy_percentage = r2 * 100

    print(f'R-squared: {r2}')
    print(f'Accuracy: {accuracy_percentage:.2f}%')
    cos_sim_list=[x*100 for x in cos_sim_list]
    return pd.concat([df_names, pd.DataFrame(cos_sim_list, columns=['resume_score(%)'])], axis=1).sort_values(by=['resume_score(%)'], ascending=False).head(10)



# Gradio interface function
def gr_interface(job_description_file):
    #gr.Warning(job_description_file.name)
    with open(job_description_file.name, 'r', encoding='utf-8') as f:
        file_desc_lst =  [r.replace('\n', '') for r in f.readlines()]
    job_description = ''
    for i in file_desc_lst:
      if len(job_description) == 0:
        job_description = str(i)
      else:
        job_description = job_description + ' ' + str(i)
    all_resume_text = []
    for i in df.iloc[:].values:
      s = ''
      for j in list(i):
        if len(s) == 0:
          s = str(j)
        else:
          s = s + ' , ' + str(j)
      all_resume_text.append(s)
    top_resumes = get_resume_score(job_description, df, all_resume_text)
    table_html=top_resumes.reset_index().to_html(index=False)
    return table_html
iface = gr.Interface(
    fn=gr_interface,
    inputs=gr.File(label="Upload Job Description", type="filepath"),
    outputs=gr.HTML(label="Top 10 Resumes"),
    theme="huggingface",
    live=True,
    title="Resume Scorer",
    description="Upload a job description file and get the top 10 matching resumes.",
    css="""
    .gr-outer-container {
        max-width: 800px;
        margin: auto;
        background-color: #f4f4f4;  /* Set your desired background color */
        font-family: 'Arial', sans-serif;  /* Set your desired font */
    }
    .gr-interface {
        padding: 20px;
        border: 1px solid #ccc;
        border-radius: 10px;
        box-shadow: 0px 0px 10px 0px #ccc;  /* Add box shadow for a nice effect */
    }
    .gr-model {
        margin-bottom: 20px;
    }
    .gr-output {
        background-color: #fff;  /* Set the background color of the output area */
        padding: 10px;
        border-radius: 5px;
    }
    .gr-model-title {
        font-size: 20px;  /* Set the font size of the model title */
        color: #333;  /* Set the font color of the model title */
    }
    """,
)

# Launch the Gradio interface
iface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:529: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-659bc3de-44ba2f1a408a0adc44e135d5;bb539244-485c-4ac9-bf28-5290b4372e62)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://168363a175834ba09f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://168363a175834ba09f.gradio.live


In [35]:

cos_sim_list=[x*100 for x in cos_sim_list]
print(cos_sim_list)

[23.440361546924777, 31.738399570961473, 27.024925097036306, 39.51703807775077, 28.56371594378868, 19.88980632395388, 30.622360094038115, 30.749688543842947, 26.79456508228335, 31.132471299766255, 24.552509967066275, 23.45771835163323, 24.880667576405973, 32.68228676411534, 29.773257907842176, 26.79456508228335, 32.16661000784325, 32.48296561496588, 32.16661000784325, 29.050921568102517, 35.90790009701612, 30.26137663344013, 11.483385035264293, 20.29994857352876, 24.45997952351143, 30.690637458832846, 30.195279627036275, 29.667616210205082, 27.48863996369953, 24.88066757640597, 36.72323309018215, 28.729720245711164, 34.046646812485946, 31.126587812744095, 23.204774044612865, 27.968818034639625, 25.15883608132604, 33.862655057584796, 24.880667576405973, 29.834709485930823, 17.36124629286286, 34.254666446809466, 30.472470011002212, 30.472470011002212, 42.52619471630004, 35.44891715150073, 24.309763284832524, 29.665411341099425, 33.45963418131047, 20.577736110739288, 30.675477844443456, 2